In [2]:
import pandas as pd
import numpy as np

dfN = pd.read_csv('NaturalFires/NaturalFires.csv')
print(len(dfN))

df_list = []
idx = 0
for nf in range(len(dfN)):
    dfhC = pd.read_csv(f'OUTPUTFWI/fw_{idx+1}.csv')
    dfhC = dfhC.iloc[48:]

    dfhC['hourly.time'] = pd.to_datetime(dfhC['hourly.time'])

    dfhC["location"] = idx+1

    df_list.append(dfhC)

    idx += 1

df = pd.concat(df_list)


print("Number of unique locations:", df['location'].nunique())

#df = df.drop(['latitude','hourly.is_day','longitude','elevation','hourly.snowfall', 'hourly.snow_depth', 'hourly.weather_code'], axis=1)

df.to_csv('concatenated_df.csv', index=False)

#df_numeric = df.select_dtypes(include=[np.number])
#correlations = df_numeric.corrwith(df_numeric['fwix']).sort_values(ascending=False)

#print(df.columns)

1770
Number of unique locations: 1770


In [5]:
from more_itertools import last
import pingouin as pg
import pandas as pd
import numpy as np

df = pd.read_csv('concatenated_df.csv')
df['hourly.time'] = pd.to_datetime(df['hourly.time'])

grouped = df.groupby('location')

partial_corrs_dict = {}

for name, group in grouped:
     
     group = group.sort_values('hourly.time')
     group.set_index('hourly.time', inplace=True)
    
     for days in [8, 16, 30]:
        last_days = group.loc[group.index.max() - pd.Timedelta(days=days):group.index.max()]
        print(last_days)
        
        '''
        variables = last_days.select_dtypes(include=[np.number]).columns
        partial_corrs = pd.Series(index=variables)

        for var in variables:
            if var != 'fwix':
                covars = [v for v in variables if v != 'fwix' and v != var]
                print(f"Calculating partial correlation for {var} with covariates {covars}")
                print(f"Values for {var}: {last_days[var].values}")
                print(f"Values for 'fwix': {last_days['fwix'].values}")
                partial_corr = pg.partial_corr(data=last_days, x='fwix', y=var, covar=covars)
                print(f"Partial correlation: {partial_corr['r'][0]}")
                partial_corrs[var] = partial_corr['r'][0]
        
        partial_corrs_dict[(name, days)] = partial_corrs

partial_corrs_df = pd.DataFrame(partial_corrs_dict)
'''

                     index  Unnamed: 0  hourly.temperature_2m  \
hourly.time                                                     
2001-07-12 23:00:00    192         192                   21.4   
2001-07-12 23:00:00    192         192                   21.4   
2001-07-12 23:00:00    192         192                   21.4   
2001-07-12 23:00:00    192         192                   21.4   
2001-07-12 23:00:00    192         192                   21.4   
...                    ...         ...                    ...   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   

                     hourly.relative_humidity_2m  hourly.dew_point_2m  \
hourly.time                                                             
2001-07-

In [9]:
df = pd.read_csv('concatenated_df.csv')
grouped = df.groupby('location')

group = grouped.get_group(1)

last_days = group.tail(24)

print(last_days)

In [7]:
import pingouin as pg
import pandas as pd
import numpy as np

df = pd.read_csv('concatenated_df.csv')
df['hourly.time'] = pd.to_datetime(df['hourly.time'])

grouped = df.groupby('location')

group = grouped.get_group(1)

group = group.sort_values('hourly.time')
group.set_index('hourly.time', inplace=True)

# Select last 8 days based on number of rows
days = 8
last_days = group.tail(days*24)

variables = last_days.select_dtypes(include=[np.number]).columns
partial_corrs = pd.Series(index=variables)

for var in variables:
    if var != 'fwix':
        covars = [v for v in variables if v != 'fwix' and v != var]
        print(f"Calculating partial correlation for {var} with covariates {covars}")
        print(f"Values for {var}: {last_days[var].values}")
        print(f"Values for 'fwix': {last_days['fwix'].values}")
        partial_corr = pg.partial_corr(data=last_days, x='fwix', y=var, covar=covars)
        print(f"Partial correlation: {partial_corr['r'][0]}")
        partial_corrs[var] = partial_corr['r'][0]

Calculating partial correlation for index with covariates ['Unnamed: 0', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.dew_point_2m', 'hourly.apparent_temperature', 'hourly.precipitation', 'hourly.rain', 'hourly.pressure_msl', 'hourly.surface_pressure', 'hourly.cloud_cover', 'hourly.cloud_cover_low', 'hourly.cloud_cover_mid', 'hourly.cloud_cover_high', 'hourly.et0_fao_evapotranspiration', 'hourly.vapour_pressure_deficit', 'hourly.wind_speed_10m', 'hourly.wind_speed_100m', 'hourly.wind_direction_10m', 'hourly.wind_direction_100m', 'hourly.wind_gusts_10m', 'hourly.soil_temperature_0_to_7cm', 'hourly.soil_temperature_7_to_28cm', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_temperature_100_to_255cm', 'hourly.soil_moisture_0_to_7cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.soil_moisture_28_to_100cm', 'hourly.soil_moisture_100_to_255cm', 'hourly.sunshine_duration', 'hourly.shortwave_radiation', 'hourly.direct_radiation', 'hourly.diffuse_radiation', 'hourly.direct_n

/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:866: RuntimeWarning: divide by zero encountered in divide
  D = np.diag(np.sqrt(1 / Vi_diag))
/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:867: RuntimeWarning: invalid value encountered in matmul
  pcor = -1 * (D @ Vi @ D)  # Partial correlation matrix
/tmp/ipykernel_3232/1354126493.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Partial correlation: {partial_corr['r'][0]}")
/tmp/ipykernel_3232/1354126493.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  partial_corrs[var] = partial_corr['r'][0]
/home/ori/.l

Partial correlation: -1.0
Calculating partial correlation for hourly.wind_direction_10m with covariates ['index', 'Unnamed: 0', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.dew_point_2m', 'hourly.apparent_temperature', 'hourly.precipitation', 'hourly.rain', 'hourly.pressure_msl', 'hourly.surface_pressure', 'hourly.cloud_cover', 'hourly.cloud_cover_low', 'hourly.cloud_cover_mid', 'hourly.cloud_cover_high', 'hourly.et0_fao_evapotranspiration', 'hourly.vapour_pressure_deficit', 'hourly.wind_speed_10m', 'hourly.wind_speed_100m', 'hourly.wind_direction_100m', 'hourly.wind_gusts_10m', 'hourly.soil_temperature_0_to_7cm', 'hourly.soil_temperature_7_to_28cm', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_temperature_100_to_255cm', 'hourly.soil_moisture_0_to_7cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.soil_moisture_28_to_100cm', 'hourly.soil_moisture_100_to_255cm', 'hourly.sunshine_duration', 'hourly.shortwave_radiation', 'hourly.direct_radiation', 'hourly.diffuse_ra

/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:866: RuntimeWarning: divide by zero encountered in divide
  D = np.diag(np.sqrt(1 / Vi_diag))
/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:867: RuntimeWarning: invalid value encountered in matmul
  pcor = -1 * (D @ Vi @ D)  # Partial correlation matrix
/tmp/ipykernel_3232/1354126493.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Partial correlation: {partial_corr['r'][0]}")
/tmp/ipykernel_3232/1354126493.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  partial_corrs[var] = partial_corr['r'][0]
/home/ori/.l

Partial correlation: nan
Calculating partial correlation for hourly.diffuse_radiation with covariates ['index', 'Unnamed: 0', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.dew_point_2m', 'hourly.apparent_temperature', 'hourly.precipitation', 'hourly.rain', 'hourly.pressure_msl', 'hourly.surface_pressure', 'hourly.cloud_cover', 'hourly.cloud_cover_low', 'hourly.cloud_cover_mid', 'hourly.cloud_cover_high', 'hourly.et0_fao_evapotranspiration', 'hourly.vapour_pressure_deficit', 'hourly.wind_speed_10m', 'hourly.wind_speed_100m', 'hourly.wind_direction_10m', 'hourly.wind_direction_100m', 'hourly.wind_gusts_10m', 'hourly.soil_temperature_0_to_7cm', 'hourly.soil_temperature_7_to_28cm', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_temperature_100_to_255cm', 'hourly.soil_moisture_0_to_7cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.soil_moisture_28_to_100cm', 'hourly.soil_moisture_100_to_255cm', 'hourly.sunshine_duration', 'hourly.shortwave_radiation', 'hourly.direct_rad

/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:866: RuntimeWarning: divide by zero encountered in divide
  D = np.diag(np.sqrt(1 / Vi_diag))
/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:867: RuntimeWarning: invalid value encountered in matmul
  pcor = -1 * (D @ Vi @ D)  # Partial correlation matrix
/home/ori/.local/lib/python3.10/site-packages/pingouin/correlation.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  tval = r * np.sqrt(dof / (1 - r**2))
/home/ori/.local/lib/python3.10/site-packages/pingouin/effsize.py:152: RuntimeWarning: divide by zero encountered in arctanh
  z = np.arctanh(stat)  # R-to-z transform
/tmp/ipykernel_3232/1354126493.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Partial correlation: {partial_c

In [8]:
print(group.tail(10))

# Select last 8 days based on number of rows
days = 8
last_days = group.tail(days*24)

print(last_days.head(10))

                     index  Unnamed: 0  hourly.temperature_2m  \
hourly.time                                                     
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   
2001-07-20 23:00:00      0           0                   19.0   

                     hourly.relative_humidity_2m  hourly.dew_point_2m  \
hourly.time                                                             
2001-07-20 23:00:00                           61                 11.2   


In [6]:
print("Number of unique locations:", df['location'].nunique())

Number of unique locations: 1


In [3]:
print(partial_corrs_df)

                                                1                    
                                               8         16        30
index                                   -0.019360 -0.040676 -0.011128
Unnamed: 0                              -0.019360 -0.040676 -0.011128
hourly.temperature_2m                   -0.240446 -0.225813 -0.179900
hourly.relative_humidity_2m             -0.322950 -0.451248 -0.361237
hourly.dew_point_2m                      0.189296  0.208151  0.152852
hourly.apparent_temperature              0.172429  0.157478  0.149236
hourly.precipitation                    -0.051118 -0.037217 -0.037969
hourly.rain                             -0.051118 -0.037217 -0.037969
hourly.pressure_msl                     -0.023514 -0.019669 -0.000618
hourly.surface_pressure                  0.020634  0.024431  0.001078
hourly.cloud_cover                       0.029782 -0.001876  0.051130
hourly.cloud_cover_low                  -0.021217  0.005832 -0.042693
hourly.cloud_cover_m

In [6]:
def calculate_partial_correlation(df, shift_by):
    # Shift the data by the specified number of rows
    df_shifted = df.shift(shift_by)

    # Drop the rows with NaN values that were created by shifting
    df_shifted = df_shifted.dropna()

    # Calculate the correlation of the shifted data with the original data
    df_numeric = df.select_dtypes(include=[np.number])
    df_numeric_shifted = df_shifted.select_dtypes(include=[np.number])
    correlations_shifted = df_numeric.corrwith(df_numeric_shifted).sort_values(ascending=False)

    return correlations_shifted

In [7]:
from sklearn.ensemble import RandomForestRegressor

for days in [8, 16, 30]:
    hours = days * 24
    print(f"Correlations for {days} days before the occurrence of the wildfire:")
    print(calculate_partial_correlation(df, hours))
    '''
    
    # Shift the data by the specified number of rows
    df_shifted = df.shift(hours)

    # Drop the rows with NaN values that were created by shifting
    df_shifted = df_shifted.dropna()

    # Drop datetime columns
    df_shifted = df_shifted.select_dtypes(exclude=[np.datetime64])

    # Assume df_shifted is your dataframe
    X = df_shifted.drop('fwix', axis=1)
    y = df_shifted['fwix']

    model = RandomForestRegressor(random_state=1451)
    model.fit(X, y)

    importances = pd.DataFrame({
        'feature': X.columns,
        'importance': np.round(model.feature_importances_, 3)
    })
    importances = importances.sort_values('importance', ascending=False)

    print("\nFeature Importances:\n", importances)
    print("\n")
    '''

Correlations for 8 days before the occurrence of the wildfire:


MemoryError: Unable to allocate 50.4 GiB for an array with shape (6766724160,) and data type int64

In [8]:
import pingouin as pg
import pandas as pd
import numpy as np

df['hourly.time'] = pd.to_datetime(df['hourly.time'])

grouped = df.groupby('location').resample('D', on='hourly.time').mean()

partial_corrs_dict = {}

for name, group in grouped:
    location = name[0]
    date = name[1]
    
    variables = group.columns
    partial_corrs = pd.Series(index=variables)

    batch_size = 10

    num_batches = int(np.ceil(len(variables) / batch_size))

    for i in range(num_batches):
        batch_vars = variables[i*batch_size:(i+1)*batch_size]
        
        for var in batch_vars:
            if var != 'fwix':
                covars = [v for v in variables if v != 'fwix' and v != var]
                partial_corr = pg.partial_corr(data=group, x='fwix', y=var, covar=covars)
                partial_corrs[var] = partial_corr['r'][0]
    
    partial_corrs_dict[(location, date)] = partial_corrs

partial_corrs_df = pd.DataFrame(partial_corrs_dict)

print(partial_corrs_df)

ModuleNotFoundError: No module named 'pingouin'